# Plotting routine for the June2 2022 Event - PSP

In [1]:
test = 'test'

In [2]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocessing
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

#### setting the times for py3DCORE

In [3]:
t_launch = datetime.datetime(2022, 6, 2, 7, 30, tzinfo=datetime.timezone.utc)

t_s = datetime.datetime(2022, 6, 2, 11, 45, tzinfo=datetime.timezone.utc)
t_e = datetime.datetime(2022, 6, 2, 16, tzinfo=datetime.timezone.utc)

t_fit = [
        datetime.datetime(2022, 6, 2, 12, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 13, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 13, 30, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 14, tzinfo=datetime.timezone.utc)
     ]


In [4]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['06/02/2022, 12:00:00', '06/02/2022, 13:00:00', '06/02/2022, 13:30:00', '06/02/2022, 14:00:00']


#### loading the pickle files

In [5]:
path = 'output/psp02062022_512_final_1/'

filepath = fp.loadpickle(path, number=-1)

In [6]:
filepath[:-7]

'output/psp02062022_512_final_1/10'

#### extracting parameters

In [7]:
res_mineps, res_mean, res_std, ind, keys, allres = fp.get_params(filepath, give_mineps=True)

print(ind)

INFO:py3dcore.fluxplot:Retrieved the following parameters for the run with minimum epsilon:
INFO:py3dcore.fluxplot: --cme_longitude -24.45
INFO:py3dcore.fluxplot: --cme_latitude 6.69
INFO:py3dcore.fluxplot: --cme_inclination 335.34
INFO:py3dcore.fluxplot: --cme_diameter_1au 0.34
INFO:py3dcore.fluxplot: --cme_aspect_ratio 1.06
INFO:py3dcore.fluxplot: --cme_launch_radius 8.17
INFO:py3dcore.fluxplot: --cme_launch_velocity 434.11
INFO:py3dcore.fluxplot: --t_factor -244.96
INFO:py3dcore.fluxplot: --cme_expansion_rate 1.14
INFO:py3dcore.fluxplot: --magnetic_decay_rate 1.64
INFO:py3dcore.fluxplot: --magnetic_field_strength_1au 10.76
INFO:py3dcore.fluxplot: --background_drag 1.23
INFO:py3dcore.fluxplot: --background_velocity 197.31


Results :

	cme_longitude: 		-36.07 +/- 8.01
	cme_latitude: 		9.84 +/- 5.81
	cme_inclination: 		338.78 +/- 5.75
	cme_diameter_1au: 		0.32 +/- 0.02
	cme_aspect_ratio: 		1.35 +/- 0.26
	cme_launch_radius: 		10.85 +/- 1.44
	cme_launch_velocity: 		408.44 +/- 125.18
	t_factor: 		-260.24 +/- 27.10
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		8.73 +/- 1.24
	background_drag: 		1.10 +/- 0.48
	background_velocity: 		367.77 +/- 158.05
4


In [8]:
out = fp.get_ensemble_stats(filepath)
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,expansion rate,B decay rate,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.00,512.00,512.000000,512.000000,512.000000
mean,-36.069935,9.837805,338.777161,0.323380,1.347182,10.846144,408.443726,-260.240295,1.14,1.64,8.726523,1.100552,367.765076
std,8.019708,5.816406,5.758680,0.020444,0.255500,1.441718,125.298233,27.129086,0.00,0.00,1.241069,0.484978,158.204269
min,-54.179661,-3.196850,318.948029,0.229214,1.000650,6.135501,203.003799,-299.976807,1.14,1.64,6.449694,0.202163,101.102875
25%,-42.277433,5.507911,335.213226,0.313302,1.159305,9.921862,310.859756,-282.200180,1.14,1.64,7.822046,0.740146,228.825043
50%,-36.416973,9.937199,338.966751,0.326850,1.296523,10.847026,385.140472,-263.277908,1.14,1.64,8.514116,1.114966,371.499039
75%,-30.616042,14.235440,342.428314,0.339797,1.481987,11.800709,493.465370,-240.602924,1.14,1.64,9.537406,1.491406,493.534172
max,-12.814528,23.246788,354.591339,0.349886,2.567653,14.812379,783.157593,-175.054337,1.14,1.64,12.850999,1.982472,692.134644


In [9]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [10]:
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    f.write('Summary of results')
    f.write('\n\n')
    f.write('Results:')
    f.write('\n')
    for i in range(1, len(keys)):    
        f.write("\t{}: \t\t{:.02f} +/- {:.02f}".format(keys[i], res_mean[i], res_std[i]))
        f.write('\n')
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for i in range(1, len(keys)):
        f.write(" --{} {:.2f}".format(keys[i], res_mineps[i]))
        f.write('\n')
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')

#### loading and plotting the fitting results

In [11]:
t_s = datetime.datetime(2022, 6, 2, 9, tzinfo=datetime.timezone.utc)
t_e = datetime.datetime(2022, 6, 2, 18, tzinfo=datetime.timezone.utc)

In [12]:
fp.fullinsitu(observer='PSP', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='SPP_RTN', best=True, 
              ensemble=True, save_fig=True, legend=True, max_index=512, title=True, fit_points=True)

2022-06-02 09:00:00+00:00 2022-06-02 18:00:00+00:00


INFO:py3dcore.fluxplot:Using HelioSat to retrieve observer data
INFO:py3dcore.fluxplot:Using parameters for run with minimum eps.


Results :

	cme_longitude: 		-36.07 +/- 8.01
	cme_latitude: 		9.84 +/- 5.81
	cme_inclination: 		338.78 +/- 5.75
	cme_diameter_1au: 		0.32 +/- 0.02
	cme_aspect_ratio: 		1.35 +/- 0.26
	cme_launch_radius: 		10.85 +/- 1.44
	cme_launch_velocity: 		408.44 +/- 125.18
	t_factor: 		-260.24 +/- 27.10
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		8.73 +/- 1.24
	background_drag: 		1.10 +/- 0.48
	background_velocity: 		367.77 +/- 158.05
540
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]


In [13]:
fp.scatterparams(filepath)

Results :

	cme_longitude: 		-36.07 +/- 8.01
	cme_latitude: 		9.84 +/- 5.81
	cme_inclination: 		338.78 +/- 5.75
	cme_diameter_1au: 		0.32 +/- 0.02
	cme_aspect_ratio: 		1.35 +/- 0.26
	cme_launch_radius: 		10.85 +/- 1.44
	cme_launch_velocity: 		408.44 +/- 125.18
	t_factor: 		-260.24 +/- 27.10
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		8.73 +/- 1.24
	background_drag: 		1.10 +/- 0.48
	background_velocity: 		367.77 +/- 158.05


In [14]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

-36.069935
8.019708
1.4417179
125.29823


#### Plotting the fluxrope and spacecraft positions

In [15]:
#measurement times 
#tm0 =  datetime.datetime(2022, 6, 2, 6)
tm1 =  t_launch + datetime.timedelta(hours=5)
#tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['psp'], planetlist=['Earth'], t=tm1, filepath=filepath, legend=False, view_azim=0, 
                view_elev=45, view_radius=0.06, index=0)

INFO:py3dcore.fluxplot:Using HelioSat to retrieve observer data


Results :

	cme_longitude: 		-36.07 +/- 8.01
	cme_latitude: 		9.84 +/- 5.81
	cme_inclination: 		338.78 +/- 5.75
	cme_diameter_1au: 		0.32 +/- 0.02
	cme_aspect_ratio: 		1.35 +/- 0.26
	cme_launch_radius: 		10.85 +/- 1.44
	cme_launch_velocity: 		408.44 +/- 125.18
	t_factor: 		-260.24 +/- 27.10
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		8.73 +/- 1.24
	background_drag: 		1.10 +/- 0.48
	background_velocity: 		367.77 +/- 158.05
{'cme_longitude': {'maximum': -88.54611206054688, 'minimum': -90.54611206054688}, 'cme_latitude': {'maximum': 4.090841293334961, 'minimum': 2.090841293334961}, 'cme_inclination': {'maximum': 336.33984375, 'minimum': 334.33984375}, 'cme_diameter_1au': {'maximum': 0.346340606212616, 'minimum': 0.32634060621261596}, 'cme_aspect_ratio': {'maximum': 1.1594062805175782, 'minimum': 0.9594062805175781}, 'cme_launch_radius': {'distribution': 'uniform', 'maximum': 8.26515064239502, 'minimum': 8.06515064239502}, 'cme

### make fluxrope 3d movie

In [ ]:
def make_frame(k):
    tm0 = t_launch
    t = tm0 + k*datetime.timedelta(hours=8)
    
    fig = fp.full3d(spacecraftlist=['psp'], planetlist=['Earth'],
              t=t, filepath=filepath, save_fig=True)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/' + frmstr + '.jpg', dpi=300)  
    return fig
    

In [ ]:
inn = [i for i in range(100)]


mpool = multiprocessing.Pool(processes=2)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=2)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=2)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=2)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
fig = fp.full3d_multiview(tm1, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

